# 1. TEST PERFORMANCE

In [ ]:
import pickle
import pandas as pd
import numpy as np
from numpy import nan
from os import listdir
from os.path import isfile, join
from os import listdir
import re

import warnings
warnings.filterwarnings("ignore")

In [2]:
fields_path = '../../data/training_validation_2/fields.csv'
fields_df = pd.read_csv(fields_path)
fields_df.columns = ['name', 'f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6']

fields_dict = {}

for idx in range(fields_df.shape[0]):
    name = fields_df.loc[idx, 'name']

    _fields = []

    for f in fields_df.columns[1:]:
        if not (str(fields_df.loc[idx, f]) == 'nan'):
            _fields.append(name + "_" + str(fields_df.loc[idx, f]))

    fields_dict[idx] = {'name': fields_df.loc[idx, 'name'], 'fields': _fields}
    
fields_dict

{0: {'name': 'CpuTemperature',
  'fields': ['CpuTemperature_vMax',
   'CpuTemperature_vMin',
   'CpuTemperature_vStd',
   'CpuTemperature_value']},
 1: {'name': 'DurationPickToPick',
  'fields': ['DurationPickToPick_vCnt',
   'DurationPickToPick_vFreq',
   'DurationPickToPick_vMax',
   'DurationPickToPick_vMin',
   'DurationPickToPick_vStd',
   'DurationPickToPick_vTrend',
   'DurationPickToPick_value']},
 2: {'name': 'DurationRobotFromFeederToTestBench',
  'fields': ['DurationRobotFromFeederToTestBench_vCnt',
   'DurationRobotFromFeederToTestBench_vFreq',
   'DurationRobotFromFeederToTestBench_vMax',
   'DurationRobotFromFeederToTestBench_vMin',
   'DurationRobotFromFeederToTestBench_vStd',
   'DurationRobotFromFeederToTestBench_vTrend',
   'DurationRobotFromFeederToTestBench_value']},
 3: {'name': 'DurationRobotFromTestBenchToFeeder',
  'fields': ['DurationRobotFromTestBenchToFeeder_vCnt',
   'DurationRobotFromTestBenchToFeeder_vFreq',
   'DurationRobotFromTestBenchToFeeder_vMax',
  

In [3]:
with open('../../data/results/models/_sub_fields_dict.pickle', 'wb') as handle:
    pickle.dump(fields_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
# Get class id and run id from filename
def parse_class_name(fname):
    p = re.compile("^class[^\d]*(\d+)_(\d+).*.csv")
    m = p.match(fname)

    return m.groups()

In [5]:
def impute_df(df):

    df = df.interpolate(limit_direction='both')

    return df

In [6]:
# Load one data file and return in a data frame
def load_data_file(path, fname):
    fullpath = join(path, fname)
    df = pd.read_csv(fullpath)
    df.columns = ['name', 'data']

    dfx = []

    for f in fields_dict:
        name = fields_dict[f]['name']
        fields = fields_dict[f]['fields']

        data = eval(df.loc[f, 'data'])  # convert data to array

        new_df = pd.DataFrame(data)
        if (f == 33) and (new_df.shape[1] == 6):  # NumberFuseDetected has a special case!
            new_df[6] = new_df[5]
            new_df[5] = np.NaN

        new_df.columns = fields_dict[f]['fields']

        dfx.append(new_df)

    merged_df = pd.concat(dfx, axis=1)  # Merge columns

    # # Do some imputation on the data file
    # merged_df = impute_df(merged_df.copy())

    c, r = parse_class_name(fname)  # Get class id and run id

    # Add class labels and run id
    merged_df['class'] = int(c)
    merged_df['run'] = int(r)

    return merged_df

In [7]:
# Load data files from a directory and return merged data frame
def load_data_files(path):
    print("In", path)
    files = []
    for f in listdir(path):
        if (isfile(join(path, f)) and (f.startswith("class"))):
            files.append(f)

    data_df_list = []
    for fname in files:
        print("Loading:", fname)

        df = load_data_file(path, fname)

        data_df_list.append(df)

    data_df = pd.concat(data_df_list, axis=0)  # Merge data frames

    return data_df

In [8]:
data_df_1 = load_data_files("../../data/training_validation_1/")
data_df_2 = load_data_files("../../data/training_validation_2/")
data_df_3 = load_data_files("../../data/ModelRefinement/")

In ../../data/training_validation_1/
Loading: class_ 0_0_data.csv
Loading: class_ 0_11_data.csv
Loading: class_ 0_17_data.csv
Loading: class_ 0_18_data.csv
Loading: class_ 0_19_data.csv
Loading: class_ 0_1_data.csv
Loading: class_ 0_20_data.csv
Loading: class_ 0_23_data.csv
Loading: class_ 0_25_data.csv
Loading: class_ 0_27_data.csv
Loading: class_ 0_29_data.csv
Loading: class_ 0_31_data.csv
Loading: class_ 0_32_data.csv
Loading: class_ 0_35_data.csv
Loading: class_ 0_36_data.csv
Loading: class_ 0_37_data.csv
Loading: class_ 0_38_data.csv
Loading: class_ 0_41_data.csv
Loading: class_ 0_43_data.csv
Loading: class_ 0_45_data.csv
Loading: class_ 0_46_data.csv
Loading: class_ 0_47_data.csv
Loading: class_ 0_53_data.csv
Loading: class_ 0_54_data.csv
Loading: class_ 0_55_data.csv
Loading: class_ 0_56_data.csv
Loading: class_ 0_57_data.csv
Loading: class_ 0_5_data.csv
Loading: class_ 0_60_data.csv
Loading: class_ 0_66_data.csv
Loading: class_ 0_67_data.csv
Loading: class_ 0_6_data.csv
Loading

In [9]:
data_df = pd.concat([data_df_1, data_df_2, data_df_3], axis=0, ignore_index=True)
df = data_df.copy(deep=True)
df

,CpuTemperature_vMax,CpuTemperature_vMin,CpuTemperature_vStd,CpuTemperature_value,DurationPickToPick_vCnt,DurationPickToPick_vFreq,DurationPickToPick_vMax,DurationPickToPick_vMin,DurationPickToPick_vStd,DurationPickToPick_vTrend,...,VacuumValveClosed_vMin,VacuumValveClosed_vStd,VacuumValveClosed_vTrend,VacuumValveClosed_value,ValidFrame_vCnt,ValidFrame_vFreq,ValidFrameOptrisPIIRCamera_vCnt,ValidFrameOptrisPIIRCamera_vFreq,class,run
0,56.75,44.75,3.523729,47.833333,0.0,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,89.0,11.457455,259.0,27.582899,0,0
1,48.75,42.25,2.395308,45.000000,2.0,0.198207,3.192,2.807,0.192500,0.3850,...,-0.715812,0.112918,-0.013857,-0.580892,114.0,11.299041,273.0,27.058227,0,0
2,47.50,42.00,2.085815,43.825000,3.0,0.296778,3.230,3.106,0.051674,0.0200,...,-0.721671,0.109421,-0.008111,-0.596008,114.0,11.277551,272.0,26.907842,0,0
3,48.25,42.25,1.853375,45.200000,3.0,0.298541,3.307,3.103,0.085391,0.0310,...,-0.745598,0.116748,-0.002082,-0.595118,114.0,11.344572,271.0,26.968238,0,0
4,50.00,42.50,2.661766,45.700000,3.0,0.298554,3.242,3.153,0.038577,-0.0360,...,-0.720206,0.112639,-0.001516,-0.615755,113.0,11.245550,271.0,26.969415,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57966,62.00,55.00,1.920449,58.175000,4.0,0.398837,3.338,3.150,0.071434,-0.0326,...,-0.774895,0.114042,-0.006810,-0.660989,113.0,11.290777,270.0,26.977955,12,3
57967,61.75,56.00,1.902137,58.525000,2.0,0.199656,3.139,3.102,0.018500,-0.0370,...,-0.803217,0.136381,-0.010167,-0.671295,113.0,11.269392,270.0,26.926866,12,3
57968,61.75,55.25,2.064582,58.750000,3.0,0.299187,4.818,3.176,0.773340,-0.8210,...,-0.825190,0.128678,0.008731,-0.684622,114.0,11.366857,270.0,26.921505,12,3
57969,63.25,55.75,2.242271,57.833333,2.0,0.199835,3.375,3.263,0.056000,-0.1120,...,-0.764153,0.150226,0.061184,-0.660927,113.0,11.280370,270.0,26.953136,12,3


In [10]:
print(df.isnull().sum().any())
# len(df[df.isna().any(axis=1)])

True


In [11]:
droped_cols = []
first_cols = df.columns.tolist()
df = df.dropna(thresh=int(df.shape[0] * 0.7), axis=1)  # Drop column if it does not have at least x values that are **not** NaN
print("col: ", df.shape)
droped_cols.extend(list(set(first_cols).difference(df.columns)))

col:  (57971, 218)


In [12]:
print(droped_cols)

['NumberFuseDetected_vMax', 'NumberFuseEstimated_value', 'IntensityTotalImage_value', 'NumberFuseEstimated_vMax', 'FuseHeatSlopeOK_vMin', 'NumberFuseEstimated_vMin', 'FeederBackgroundIlluminationIntensity_value', 'FeederBackgroundIlluminationIntensity_vTrend', 'SharpnessImage_vMax', 'FuseHeatSlopeOK_value', 'FeederBackgroundIlluminationIntensity_vMin', 'SharpnessImage_value', 'FuseHeatSlopeOK_vTrend', 'SharpnessImage_vTrend', 'IntensityTotalImage_vMax', 'FuseHeatSlopeOK_vMax', 'SharpnessImage_vStd', 'IntensityTotalImage_vTrend', 'IntensityTotalImage_vMin', 'FeederBackgroundIlluminationIntensity_vStd', 'FeederBackgroundIlluminationIntensity_vMax', 'FuseHeatSlopeOK_vStd', 'NumberFuseDetected_vTrend', 'NumberFuseDetected_value', 'NumberFuseDetected_vStd', 'IntensityTotalImage_vStd', 'NumberFuseDetected_vMin', 'SharpnessImage_vMin', 'NumberFuseEstimated_vStd', 'FuseHeatSlopeNOK_vTrend', 'NumberFuseEstimated_vTrend']


In [13]:
def fill_nan_values(data, name, fields):

    field_df = data[fields]

    if field_df.isnull().values.any():
        data[fields] = field_df.interpolate(method='linear', limit_direction='both')

    return data[fields]

In [14]:
for f in fields_dict:

    name = fields_dict[f]['name']
    fields = fields_dict[f]['fields']

    print("\nname:", name, "fields:", fields)
    fields = list(set(fields).difference(droped_cols))
    df_ = df.groupby(["class", "run"]).apply(fill_nan_values, name, fields)
    df_.reset_index(drop=True, inplace=True)
    df[fields] = df_[fields]


name: CpuTemperature fields: ['CpuTemperature_vMax', 'CpuTemperature_vMin', 'CpuTemperature_vStd', 'CpuTemperature_value']

name: DurationPickToPick fields: ['DurationPickToPick_vCnt', 'DurationPickToPick_vFreq', 'DurationPickToPick_vMax', 'DurationPickToPick_vMin', 'DurationPickToPick_vStd', 'DurationPickToPick_vTrend', 'DurationPickToPick_value']

name: DurationRobotFromFeederToTestBench fields: ['DurationRobotFromFeederToTestBench_vCnt', 'DurationRobotFromFeederToTestBench_vFreq', 'DurationRobotFromFeederToTestBench_vMax', 'DurationRobotFromFeederToTestBench_vMin', 'DurationRobotFromFeederToTestBench_vStd', 'DurationRobotFromFeederToTestBench_vTrend', 'DurationRobotFromFeederToTestBench_value']

name: DurationRobotFromTestBenchToFeeder fields: ['DurationRobotFromTestBenchToFeeder_vCnt', 'DurationRobotFromTestBenchToFeeder_vFreq', 'DurationRobotFromTestBenchToFeeder_vMax', 'DurationRobotFromTestBenchToFeeder_vMin', 'DurationRobotFromTestBenchToFeeder_vStd', 'DurationRobotFromTestBen


name: Vacuum fields: ['Vacuum_vCnt', 'Vacuum_vFreq', 'Vacuum_vMax', 'Vacuum_vMin', 'Vacuum_vStd', 'Vacuum_vTrend', 'Vacuum_value']

name: VacuumFusePicked fields: ['VacuumFusePicked_vCnt', 'VacuumFusePicked_vFreq', 'VacuumFusePicked_vMax', 'VacuumFusePicked_vMin', 'VacuumFusePicked_vStd', 'VacuumFusePicked_vTrend', 'VacuumFusePicked_value']

name: VacuumValveClosed fields: ['VacuumValveClosed_vCnt', 'VacuumValveClosed_vFreq', 'VacuumValveClosed_vMax', 'VacuumValveClosed_vMin', 'VacuumValveClosed_vStd', 'VacuumValveClosed_vTrend', 'VacuumValveClosed_value']

name: ValidFrame fields: ['ValidFrame_vCnt', 'ValidFrame_vFreq']

name: ValidFrameOptrisPIIRCamera fields: ['ValidFrameOptrisPIIRCamera_vCnt', 'ValidFrameOptrisPIIRCamera_vFreq']


In [15]:
print(df.isnull().sum().any())

False


In [16]:
for column in df.columns:
    if column not in ["class", "run"]:
        if (len(df[column].unique()) == 1) or (df[column].isnull().all()):
            df.drop(column, inplace=True, axis=1)
            droped_cols.append(column)
            print(column, "droped-unique")

        else:
            zero_rows = df.loc[df[column] == float(0)]
            if zero_rows.shape[0] >= df.shape[0] * 50:
                df.drop(column, inplace=True, axis=1)
                droped_cols.append(column)
                print(column, "droped-zero")

FuseOutsideOperationalSpace_vMin droped-unique
LightBarrieActiveTaskDuration2_vCnt droped-unique
LightBarrieActiveTaskDuration2_vFreq droped-unique
LightBarrierActiveTaskDuration1b_vCnt droped-unique
LightBarrierActiveTaskDuration1b_vFreq droped-unique
LightBarrierPassiveTaskDuration1b_vCnt droped-unique
LightBarrierPassiveTaskDuration1b_vFreq droped-unique
LightBarrierPassiveTaskDuration2_vCnt droped-unique
LightBarrierPassiveTaskDuration2_vFreq droped-unique
LightBarrierTaskDuration_vCnt droped-unique
LightBarrierTaskDuration_vFreq droped-unique


In [17]:
df

,CpuTemperature_vMax,CpuTemperature_vMin,CpuTemperature_vStd,CpuTemperature_value,DurationPickToPick_vCnt,DurationPickToPick_vFreq,DurationPickToPick_vMax,DurationPickToPick_vMin,DurationPickToPick_vStd,DurationPickToPick_vTrend,...,VacuumValveClosed_vMin,VacuumValveClosed_vStd,VacuumValveClosed_vTrend,VacuumValveClosed_value,ValidFrame_vCnt,ValidFrame_vFreq,ValidFrameOptrisPIIRCamera_vCnt,ValidFrameOptrisPIIRCamera_vFreq,class,run
0,56.75,44.75,3.523729,47.833333,0.0,0.000000,3.192,2.807,0.192500,0.3850,...,-0.715812,0.112918,-0.013857,-0.580892,89.0,11.457455,259.0,27.582899,0,0
1,48.75,42.25,2.395308,45.000000,2.0,0.198207,3.192,2.807,0.192500,0.3850,...,-0.715812,0.112918,-0.013857,-0.580892,114.0,11.299041,273.0,27.058227,0,0
2,47.50,42.00,2.085815,43.825000,3.0,0.296778,3.230,3.106,0.051674,0.0200,...,-0.721671,0.109421,-0.008111,-0.596008,114.0,11.277551,272.0,26.907842,0,0
3,48.25,42.25,1.853375,45.200000,3.0,0.298541,3.307,3.103,0.085391,0.0310,...,-0.745598,0.116748,-0.002082,-0.595118,114.0,11.344572,271.0,26.968238,0,0
4,50.00,42.50,2.661766,45.700000,3.0,0.298554,3.242,3.153,0.038577,-0.0360,...,-0.720206,0.112639,-0.001516,-0.615755,113.0,11.245550,271.0,26.969415,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57966,62.00,55.00,1.920449,58.175000,4.0,0.398837,3.338,3.150,0.071434,-0.0326,...,-0.774895,0.114042,-0.006810,-0.660989,113.0,11.290777,270.0,26.977955,12,3
57967,61.75,56.00,1.902137,58.525000,2.0,0.199656,3.139,3.102,0.018500,-0.0370,...,-0.803217,0.136381,-0.010167,-0.671295,113.0,11.269392,270.0,26.926866,12,3
57968,61.75,55.25,2.064582,58.750000,3.0,0.299187,4.818,3.176,0.773340,-0.8210,...,-0.825190,0.128678,0.008731,-0.684622,114.0,11.366857,270.0,26.921505,12,3
57969,63.25,55.75,2.242271,57.833333,2.0,0.199835,3.375,3.263,0.056000,-0.1120,...,-0.764153,0.150226,0.061184,-0.660927,113.0,11.280370,270.0,26.953136,12,3


In [18]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, matthews_corrcoef
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

In [19]:
df['runId'] = 1000 * df['class'] + df['run']
run_df = df[['class', 'runId']].copy()
run_df.drop_duplicates(inplace=True)
run_df.reset_index(inplace=True)
run_df_ = df['run'].copy()
del run_df['index'], df['run']

In [20]:
importance_df = pd.read_pickle("../../data/importance_df_ba.pkl")
importance_df = importance_df.loc[(importance_df["importance_mean"] > 0)]

sorted_features_imp = list(importance_df["feature"].values)
value_features_imp = list(importance_df["importance_mean"].values)
f_imp = [(name, value) for name, value in zip(sorted_features_imp, value_features_imp)]

# print("fold_num:", len(f_imp), f_imp)
print(len(sorted_features_imp), sorted_features_imp)

141 ['Temperature_value', 'Humidity_value', 'DurationTestBenchClosed_vFreq', 'FuseHeatSlopeNOK_vMin', 'Pressure_vMax', 'FeederAction4_vCnt', 'ProcessCpuLoadNormalized_value', 'Vacuum_vFreq', 'TotalMemoryConsumption_vMin', 'CpuTemperature_vStd', 'SmartMotorPositionError_vMax', 'FuseCycleDuration_vMax', 'LightBarrierActiveTaskDuration1_vTrend', 'DurationRobotFromFeederToTestBench_vMin', 'SmartMotorSpeed_vMax', 'FusePicked_vFreq', 'DurationPickToPick_vFreq', 'DurationTestBenchClosed_value', 'DurationPickToPick_vTrend', 'VacuumValveClosed_vStd', 'LightBarrierPassiveTaskDuration1_vCnt', 'DurationRobotFromTestBenchToFeeder_vMin', 'NumberFuseEstimated_vFreq', 'VacuumFusePicked_vFreq', 'FuseTestResult_vStd', 'EPOSCurrent_vStd', 'DurationTestBenchClosed_vMin', 'VacuumFusePicked_vMin', 'TemperatureThermoCam_value', 'SmartMotorPositionError_vFreq', 'LightBarrierActiveTaskDuration1_vMax', 'IntensityTotalThermoImage_vMax', 'DurationRobotFromFeederToTestBench_vStd', 'TemperatureThermoCam_vTrend', 'E

In [21]:
# split a sequence into samples
def create_sequence(sequence, n_steps):
    X = list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x = sequence[i:end_ix]
        X.append(seq_x)

    return np.array(X)

def create_dataset_for_run(df, ws):
    #     data_data = np.empty((0, ws * len(sensor_list))) # for 1D
    #     data_data = np.empty((0, ws, len(sensor_list))) # for 2D
    #     data_data = np.empty((0, len(sensor_list), ws)) # for 2D
    #     label_data = np.empty((0, 1))

    sensors_df = df.filter(sensor_list)

    # Calculate seq of windows_size len
    seq = create_sequence(sensors_df.values, n_steps=ws)
    #     seq = np.transpose(seq, axes=(0, 2, 1))
    seq_count = seq.shape[0]
    seq = seq.reshape((seq_count, -1))  # for 1D

    # add new seq to data_data array
    # data_data = np.vstack((data_data, seq))

    # Calculate RULS
    labels = df['class'].values[:seq_count]

    # add rul to rul_data array
    #     rul_data = np.vstack((rul_data, ruls))

    # TODO: What is RUL_Max in this context?

    # print ("Shape:", seq.shape, labels.shape)

    return seq, labels

# TODO: X_t, X_tp1, y_t, y_tp1 should be calculated per run.
# TODO: Then should be merged into one X_t, X_tp1, y_t, y_tp1.

def create_datasets(df, ws):
    run_list = df['runId'].unique()
    l_len_runs = []

    X_df_list = []
    y_df_list = []

    for r in run_list:
        r_df = df[df['runId'] == r]
        # print ("--> r: ", r, r_df.shape)
        sensor_data, label_data = create_dataset_for_run(r_df, ws)

        # Post Processing for the model

        # Padding for model input
        padded_sensor_data = sensor_data.copy()  # np.hstack((sensor_data, np.zeros((sensor_data.shape[0], 2)))) # for AE

        # Calculate X(t) and X(t+1) for model input/output
        X_t = padded_sensor_data[:]

        # Calculate y(t) and y(t+1) for model input/output
        y_t = label_data[:]

        X_df_list.append(pd.DataFrame(X_t))
        y_df_list.append(pd.DataFrame(y_t))
        l_len_runs.append(len(X_t))

    X_t = pd.concat(X_df_list, axis=0)  # Merge data frames
    y_t = pd.concat(y_df_list, axis=0)  # Merge data frames

    return X_t.values, y_t.values.flatten(), run_list, l_len_runs


In [22]:
fold_num = 3
cv = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=41)


In [23]:
sensor_list = sorted_features_imp[:len(sorted_features_imp)].copy()
df_report = pd.DataFrame()

In [24]:
ws = 40

acc_sum_1 = 0
f1_sum_1 = 0
mcc_sum_1 = 0

acc_sum_4 = 0
f1_sum_4 = 0
mcc_sum_4 = 0

print("_______________________________________________________________")

for fold, (training_indices, validation_indices) in enumerate(cv.split(run_df['runId'], run_df['class'])):
    print("----------------------------------------------------------")

    report_index = "ws_{}_fold_{}".format(ws, fold + 1)
    print("Fold: ", report_index)

    training_runIds = run_df.loc[training_indices]['runId']
    validation_runIds = run_df.loc[validation_indices]['runId']

    df_ = df.loc[~(df["runId"].isin([56, 74, 49, 23, 35, 6, 83, 54]))]

    X_train_df = df_[df_['runId'].isin(training_runIds)].copy()
    X_val_df = df_[df_['runId'].isin(validation_runIds)].copy()

    X_train_df = X_train_df[sensor_list + ["class", "runId"]].copy()
    X_val_df = X_val_df[sensor_list + ["class", "runId"]].copy()

    scaler_cols = sensor_list.copy()  # list(set(sensor_list).difference(["class", "runId"]))

    scaler = RobustScaler()
    scaler_data_tr = scaler.fit_transform(X_train_df[scaler_cols])
    scaler_data_tr = pd.DataFrame(scaler_data_tr, index=X_train_df.index, columns=scaler_cols)
    X_train_df = pd.concat([X_train_df[["class", "runId"]], scaler_data_tr], axis=1)

    scaler_data_ts = scaler.transform(X_val_df[scaler_cols])
    scaler_data_ts = pd.DataFrame(scaler_data_ts, index=X_val_df.index, columns=scaler_cols)
    X_val_df = pd.concat([X_val_df[["class", "runId"]], scaler_data_ts], axis=1)

    X_train, y_train, runList_tr, l_len_runs_tr = create_datasets(X_train_df, ws)
    X_val, y_val, runList_val, l_len_runs_val = create_datasets(X_val_df, ws)

    lda = LinearDiscriminantAnalysis()
    X_train = lda.fit_transform(X_train, y_train)
    X_val = lda.transform(X_val)

    # model1 = OneVsOneClassifier(LGBMClassifier(random_state=41))
    model1 = LGBMClassifier(random_state=41)
    model1.fit(X_train, y_train)
    pred = model1.predict(X_val)

    acc_val1 = round(accuracy_score(y_val, pred), 3)
    f1_val1 = round(f1_score(y_val, pred, average='weighted'), 3)
    mcc_val1 = round(matthews_corrcoef(y_val, pred), 3)
    cm1 = confusion_matrix(y_val, pred)

    l_index = []
    for run_, num_run in zip(runList_val, l_len_runs_val):
        l_index.extend([run_] * num_run)

    # # 56(4), 74(4), 49(4); 7000
    # # 23(4), 35(4), 6(7 ama az), 83(iyi); 9001
    # # 54(9)
    # print(len(l_index), len(y_val))
    df_result = pd.DataFrame()
    df_result["actual"] = y_val
    df_result["pred"] = pred
    df_result.loc[df_result.index[0], "cm"] = str(cm1)
    df_result.index = l_index

    df_result.to_excel("../../data/results/excels/_lgb_{}_mcc.xlsx".format(report_index))
    with open('../../data/results/models/_lgb_{}_mcc.pickle'.format(report_index), 'wb') as handle:
        pickle.dump(model1, handle, protocol=pickle.HIGHEST_PROTOCOL)

    acc_sum_1 += acc_val1
    f1_sum_1 += f1_val1
    mcc_sum_1 += mcc_val1

    print(cm1)
    print("LightGBM Fold:", fold, "ACC:", acc_val1, "F1:", f1_val1, "MCC:", mcc_val1)
    df_report.loc[report_index, "LGBM_ACC"] = acc_val1
    df_report.loc[report_index, "LGBM_F1"] = f1_val1
    df_report.loc[report_index, "LGBM_MCC"] = mcc_val1

    param = {
        'objective': 'multi:softprob',  # error evaluation for multiclass training
        'num_class': len(pd.unique(y_train))}  # the number of classes that exist in this datset
    # model4 = OneVsOneClassifier(XGBClassifier(param, random_state=41, verbosity=0))
    model4 = XGBClassifier(param, random_state=41, verbosity=0)
    model4.fit(X_train, y_train)
    pred = model4.predict(X_val)

    acc_val4 = round(accuracy_score(y_val, pred), 3)
    f1_val4 = round(f1_score(y_val, pred, average='weighted'), 3)
    mcc_val4 = round(matthews_corrcoef(y_val, pred), 3)
    cm4 = confusion_matrix(y_val, pred)

    acc_sum_4 += acc_val4
    f1_sum_4 += f1_val4
    mcc_sum_4 += mcc_val4

    print(cm4)
    print("XGBClassifier Fold:", fold, "ACC:", acc_val4, "F1:", f1_val4, "MCC:", mcc_val4)
    df_report.loc[report_index, "XGB_ACC"] = acc_val4
    df_report.loc[report_index, "XGB_F1"] = f1_val4
    df_report.loc[report_index, "XGB_MCC"] = mcc_val4

    df_result = pd.DataFrame()
    df_result["actual"] = y_val
    df_result["pred"] = pred
    df_result.loc[df_result.index[0], "cm"] = str(cm4)
    df_result.index = l_index
    df_result.to_excel("../../data/results/excels/_xgb_{}_mcc.xlsx".format(report_index))
    with open('../../data/results/models/_xgb_{}_mcc.pickle'.format(report_index), 'wb') as handle:
        pickle.dump(model4, handle, protocol=pickle.HIGHEST_PROTOCOL)

ave_acc1 = round(acc_sum_1 / fold_num, 3)
ave_f1_score1 = round(f1_sum_1 / fold_num, 3)
ave_mcc1 = round(mcc_sum_1 / fold_num, 3)

ave_acc4 = round(acc_sum_4 / fold_num, 3)
ave_f1_score4 = round(f1_sum_4 / fold_num, 3)
ave_mcc4 = round(mcc_sum_4 / fold_num, 3)

print("\nLightGBM Avg ACC:", ave_acc1, "Avg F1:", ave_f1_score1, "Avg MCC:", ave_mcc1)
print("XGBM Avg ACC:", ave_acc4, "Avg F1:", ave_f1_score4, "Avg MCC:", ave_mcc4)

_______________________________________________________________
----------------------------------------------------------
Fold:  ws_40_fold_1
[[10041     4    13   188    37     1     0    68     1]
 [   19   566     0    54     0     0     0     0     4]
 [    0     0   646     0     0     0     0     0     0]
 [   96     0     0   225     0     0     0     0     0]
 [   21     1     0     0  2052     1     0     4     0]
 [  343     1     0     2     0  1023     0     1     1]
 [    0     0     0     3     0     0  1360     0     0]
 [    3     0     0     0     0     0     0  1038     0]
 [    0     1     0    52     0     0     0     0   986]]
LightGBM Fold: 0 ACC: 0.951 F1: 0.953 MCC: 0.927
[[10109    41    15   145    22     1     2    17     1]
 [   13   580     0    49     0     1     0     0     0]
 [    6     0   640     0     0     0     0     0     0]
 [   65     0     0   256     0     0     0     0     0]
 [    0     0     0     0  2078     1     0     0     0]
 [  326  

In [26]:
print(sensor_list)

['Temperature_value', 'Humidity_value', 'DurationTestBenchClosed_vFreq', 'FuseHeatSlopeNOK_vMin', 'Pressure_vMax', 'FeederAction4_vCnt', 'ProcessCpuLoadNormalized_value', 'Vacuum_vFreq', 'TotalMemoryConsumption_vMin', 'CpuTemperature_vStd', 'SmartMotorPositionError_vMax', 'FuseCycleDuration_vMax', 'LightBarrierActiveTaskDuration1_vTrend', 'DurationRobotFromFeederToTestBench_vMin', 'SmartMotorSpeed_vMax', 'FusePicked_vFreq', 'DurationPickToPick_vFreq', 'DurationTestBenchClosed_value', 'DurationPickToPick_vTrend', 'VacuumValveClosed_vStd', 'LightBarrierPassiveTaskDuration1_vCnt', 'DurationRobotFromTestBenchToFeeder_vMin', 'NumberFuseEstimated_vFreq', 'VacuumFusePicked_vFreq', 'FuseTestResult_vStd', 'EPOSCurrent_vStd', 'DurationTestBenchClosed_vMin', 'VacuumFusePicked_vMin', 'TemperatureThermoCam_value', 'SmartMotorPositionError_vFreq', 'LightBarrierActiveTaskDuration1_vMax', 'IntensityTotalThermoImage_vMax', 'DurationRobotFromFeederToTestBench_vStd', 'TemperatureThermoCam_vTrend', 'EPOSV

In [27]:
with open('../../data/results/models/_sub_sensor_list.pickle', 'wb') as handle:
    pickle.dump(sensor_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.21.3.


In [29]:
ws = 40

acc_sum_1 = 0
f1_sum_1 = 0
mcc_sum_1 = 0

print("_______________________________________________________________")

df_ = df.loc[~(df["runId"].isin([56, 74, 49, 23, 35, 6, 83, 54]))]

scaler_cols = sensor_list.copy()

scaler = RobustScaler()
scaler_data_tr = scaler.fit_transform(df_[scaler_cols])
scaler_data_tr = pd.DataFrame(scaler_data_tr, index=df_.index, columns=scaler_cols)
X_train_df = pd.concat([df_[["class", "runId"]], scaler_data_tr], axis=1)

# scaler_data_ts = scaler.transform(X_val_df[scaler_cols])
# scaler_data_ts = pd.DataFrame(scaler_data_ts, index=X_val_df.index, columns=scaler_cols)
# X_val_df = pd.concat([X_val_df[["class", "runId"]], scaler_data_ts], axis=1)

X_train, y_train, runList_tr, l_len_runs_tr = create_datasets(X_train_df, ws)
# X_val, y_val, runList_val, l_len_runs_val = create_datasets(X_val_df, ws)

lda = LinearDiscriminantAnalysis()
X_train = lda.fit_transform(X_train, y_train)
# X_val = lda.transform(X_val)

l_index = []
for run_, num_run in zip(runList_tr, l_len_runs_tr):
    l_index.extend([run_] * num_run)

model1 = LGBMClassifier(random_state=41)
model1.fit(X_train, y_train)
# pred = model1.predict(X_val)
pred = model1.predict(X_train)

acc_val1 = round(accuracy_score(y_train, pred), 3)
f1_val1 = round(f1_score(y_train, pred, average='weighted'), 3)
mcc_val1 = round(matthews_corrcoef(y_train, pred), 3)
cm1 = confusion_matrix(y_train, pred)

print(cm1)
print("LGBClassifier Fold:", fold, "ACC:", acc_val1, "F1:", f1_val1, "MCC:", mcc_val1)

df_result = pd.DataFrame()
df_result["actual"] = y_train
df_result["pred"] = pred
df_result.loc[df_result.index[0], "cm"] = str(cm1)
df_result.index = l_index

df_result.to_excel("../../data/results/excels/_sub_results_lgb.xlsx")

with open('../../data/results/models/_sub_scaler2.pickle', 'wb') as handle:
    pickle.dump(scaler, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../../data/results/models/_sub_lda2.pickle', 'wb') as handle:
    pickle.dump(lda, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../../data/results/models/_sub_model_lgb.pickle', 'wb') as handle:
    pickle.dump(model1, handle, protocol=pickle.HIGHEST_PROTOCOL)

_______________________________________________________________
[[29287     0     0     0     0     0     0     0     0]
 [    0  2679     0     0     0     0     0     0     0]
 [    0     0  2727     0     0     0     0     0     0]
 [    0     0     0  1681     0     0     0     0     0]
 [    0     0     0     0  2722     0     0     0     0]
 [    0     0     0     0     0  2734     0     0     0]
 [    0     0     0     0     0     0  2733     0     0]
 [    0     0     0     0     0     0     0  2402     0]
 [    0     0     0     0     0     0     0     0  2399]]
LGBClassifier Fold: 2 ACC: 1.0 F1: 1.0 MCC: 1.0


In [30]:
ws = 40

acc_sum_4 = 0
f1_sum_4 = 0
mcc_sum_4 = 0

print("_______________________________________________________________")

df_ = df.loc[~(df["runId"].isin([56, 74, 49, 23, 35, 6, 83, 54]))]

scaler_cols = sensor_list.copy()

scaler = RobustScaler()
scaler_data_tr = scaler.fit_transform(df_[scaler_cols])
scaler_data_tr = pd.DataFrame(scaler_data_tr, index=df_.index, columns=scaler_cols)
X_train_df = pd.concat([df_[["class", "runId"]], scaler_data_tr], axis=1)

# scaler_data_ts = scaler.transform(X_val_df[scaler_cols])
# scaler_data_ts = pd.DataFrame(scaler_data_ts, index=X_val_df.index, columns=scaler_cols)
# X_val_df = pd.concat([X_val_df[["class", "runId"]], scaler_data_ts], axis=1)

X_train, y_train, runList_tr, l_len_runs_tr = create_datasets(X_train_df, ws)
# X_val, y_val, runList_val, l_len_runs_val = create_datasets(X_val_df, ws)

lda = LinearDiscriminantAnalysis()
X_train = lda.fit_transform(X_train, y_train)
# X_val = lda.transform(X_val)

l_index = []
for run_, num_run in zip(runList_tr, l_len_runs_tr):
    l_index.extend([run_] * num_run)

param = {
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': len(pd.unique(y_train))}  # the number of classes that exist in this datset

model4 = XGBClassifier(param, random_state=41, verbosity=0)
model4.fit(X_train, y_train)
# pred = model4.predict(X_val)
pred = model4.predict(X_train)

acc_val4 = round(accuracy_score(y_train, pred), 3)
f1_val4 = round(f1_score(y_train, pred, average='weighted'), 3)
mcc_val4 = round(matthews_corrcoef(y_train, pred), 3)
cm4 = confusion_matrix(y_train, pred)

print(cm4)
print("XGBClassifier Fold:", fold, "ACC:", acc_val4, "F1:", f1_val4, "MCC:", mcc_val4)

df_result = pd.DataFrame()
df_result["actual"] = y_train
df_result["pred"] = pred
df_result.loc[df_result.index[0], "cm"] = str(cm4)
df_result.index = l_index

df_result.to_excel("../../data/results/excels/_sub_results_xgb.xlsx")

with open('../../data/results/models/_sub_scaler.pickle', 'wb') as handle:
    pickle.dump(scaler, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../../data/results/models/_sub_lda.pickle', 'wb') as handle:
    pickle.dump(lda, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../../data/results/models/_sub_model_xgb.pickle', 'wb') as handle:
    pickle.dump(model4, handle, protocol=pickle.HIGHEST_PROTOCOL)

_______________________________________________________________
[[29287     0     0     0     0     0     0     0     0]
 [    0  2679     0     0     0     0     0     0     0]
 [    0     0  2727     0     0     0     0     0     0]
 [    0     0     0  1681     0     0     0     0     0]
 [    0     0     0     0  2722     0     0     0     0]
 [    0     0     0     0     0  2734     0     0     0]
 [    0     0     0     0     0     0  2733     0     0]
 [    0     0     0     0     0     0     0  2402     0]
 [    0     0     0     0     0     0     0     0  2399]]
XGBClassifier Fold: 2 ACC: 1.0 F1: 1.0 MCC: 1.0


In [31]:
from sklearn.ensemble import RandomForestClassifier

In [32]:
ws = 40

acc_sum_2 = 0
f1_sum_2 = 0
mcc_sum_2 = 0

print("_______________________________________________________________")

df_ = df.loc[~(df["runId"].isin([56, 74, 49, 23, 35, 6, 83, 54]))]

scaler_cols = sensor_list.copy()

scaler = RobustScaler()
scaler_data_tr = scaler.fit_transform(df_[scaler_cols])
scaler_data_tr = pd.DataFrame(scaler_data_tr, index=df_.index, columns=scaler_cols)
X_train_df = pd.concat([df_[["class", "runId"]], scaler_data_tr], axis=1)

# scaler_data_ts = scaler.transform(X_val_df[scaler_cols])
# scaler_data_ts = pd.DataFrame(scaler_data_ts, index=X_val_df.index, columns=scaler_cols)
# X_val_df = pd.concat([X_val_df[["class", "runId"]], scaler_data_ts], axis=1)

X_train, y_train, runList_tr, l_len_runs_tr = create_datasets(X_train_df, ws)
# X_val, y_val, runList_val, l_len_runs_val = create_datasets(X_val_df, ws)

lda = LinearDiscriminantAnalysis()
X_train = lda.fit_transform(X_train, y_train)
# X_val = lda.transform(X_val)

l_index = []
for run_, num_run in zip(runList_tr, l_len_runs_tr):
    l_index.extend([run_] * num_run)

model2 = RandomForestClassifier(random_state=41)
model2.fit(X_train, y_train)
# pred = model1.predict(X_val)
pred = model2.predict(X_train)

acc_val2 = round(accuracy_score(y_train, pred), 3)
f1_val2 = round(f1_score(y_train, pred, average='weighted'), 3)
mcc_val2 = round(matthews_corrcoef(y_train, pred), 3)
cm2 = confusion_matrix(y_train, pred)

print(cm2)
print("RFClassifier Fold:", fold, "ACC:", acc_val2, "F1:", f1_val2, "MCC:", mcc_val2)

df_result = pd.DataFrame()
df_result["actual"] = y_train
df_result["pred"] = pred
df_result.loc[df_result.index[0], "cm"] = str(cm2)
df_result.index = l_index

df_result.to_excel("../../data/results/excels/_sub_results_rf.xlsx")

with open('../../data/results/models/_sub_scaler3.pickle', 'wb') as handle:
    pickle.dump(scaler, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../../data/results/models/_sub_lda3.pickle', 'wb') as handle:
    pickle.dump(lda, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../../data/results/models/_sub_model_rf.pickle', 'wb') as handle:
    pickle.dump(model2, handle, protocol=pickle.HIGHEST_PROTOCOL)

_______________________________________________________________
[[29285     0     0     0     0     2     0     0     0]
 [    1  2678     0     0     0     0     0     0     0]
 [    0     0  2727     0     0     0     0     0     0]
 [    0     0     0  1681     0     0     0     0     0]
 [    0     0     0     0  2722     0     0     0     0]
 [    0     0     0     0     0  2734     0     0     0]
 [    0     0     0     0     0     0  2733     0     0]
 [    0     0     0     0     0     0     0  2402     0]
 [    0     0     0     0     0     0     0     0  2399]]
RFClassifier Fold: 2 ACC: 1.0 F1: 1.0 MCC: 1.0


# 2. CLUSTERING

In [33]:
from sklearn.cluster import KMeans
from collections import Counter

In [34]:
X_train = scaler_data_tr[['Temperature_value', 'Humidity_value']]
X_train

,Temperature_value,Humidity_value
0,-1.594286,0.173765
1,-1.594286,0.173765
2,-1.594286,0.173765
3,-1.594286,0.173765
4,-1.594286,0.173765
...,...,...
57966,-0.492245,-1.047533
57967,-0.492245,-1.047533
57968,-0.494286,-1.047565
57969,-0.494694,-1.047572


In [35]:
model = KMeans(n_clusters=2).fit(X_train)
y_pred = model.labels_
Counter(y_pred)

Counter({0: 23767, 1: 29146})

In [36]:
with open('../../data/results/models/_sub_model_kmeans.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def load_pickle_files():
    
    with open("../../data/results/models/_sub_fields_dict.pickle", "rb") as input_file:
        fields_dict = pickle.load(input_file)
    
    with open("../../data/results/models/_sub_sensor_list.pickle", "rb") as input_file:
        sensor_list = pickle.load(input_file)
    
    with open("../../data/results/models/_sub_scaler.pickle", "rb") as input_file:
        scaler = pickle.load(input_file)
        
    with open("../../data/results/models/_sub_lda.pickle", "rb") as input_file:
        lda = pickle.load(input_file)
        
    with open("../../data/results/models/_sub_model_xgb.pickle", "rb") as input_file:
        model4 = pickle.load(input_file)
        
    with open("../../data/results/models/_sub_model_lgb.pickle", "rb") as input_file:
        model1 = pickle.load(input_file)
        
    with open("../../data/results/models/_sub_model_rf.pickle", "rb") as input_file:
        model2 = pickle.load(input_file)
        
    with open("../../data/results/models/_sub_model_kmeans.pickle", "rb") as input_file:
        model = pickle.load(input_file)
        
    return fields_dict, sensor_list, scaler, lda, model4, model  

In [ ]:
# fields_dict, sensor_list, scaler, lda, model4, model  = load_pickle_files()

In [37]:
with open("../../data/results/models/_sub_scaler.pickle", "rb") as input_file:
        scaler = pickle.load(input_file)
        
with open("../../data/results/models/_sub_lda.pickle", "rb") as input_file:
        lda = pickle.load(input_file)



In [38]:
l_pickles_xgb = [fields_dict, sensor_list, scaler, lda, model4, model]

with open('model_pickle/_sub_l_pickles_xgb.pickle', 'wb') as handle:
    pickle.dump(l_pickles_xgb, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [39]:
with open("../../data/results/models/_sub_scaler2.pickle", "rb") as input_file:
        scaler2 = pickle.load(input_file)
        
with open("../../data/results/models/_sub_lda2.pickle", "rb") as input_file:
        lda2 = pickle.load(input_file)

In [40]:
l_pickles_lgb = [fields_dict, sensor_list, scaler2, lda2, model1, model]

with open('model_pickle/_sub_l_pickles_lgb.pickle', 'wb') as handle:
    pickle.dump(l_pickles_lgb, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [41]:
with open("../../data/results/models/_sub_scaler3.pickle", "rb") as input_file:
        scaler3 = pickle.load(input_file)
        
with open("../../data/results/models/_sub_lda3.pickle", "rb") as input_file:
        lda3 = pickle.load(input_file)

In [42]:
l_pickles_rf = [fields_dict, sensor_list, scaler3, lda3, model2, model]

with open('model_pickle/_sub_l_pickles_rf.pickle', 'wb') as handle:
    pickle.dump(l_pickles_rf, handle, protocol=pickle.HIGHEST_PROTOCOL)